In [2]:
import findspark
findspark.init()
import pyspark
import random
sc = pyspark.SparkContext(appName="Pi")
num_samples = 1000000000
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1
count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples
print(pi)
sc.stop() 

3.141573648


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import *
from pyspark.sql.functions import sum

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1'

In [4]:
import logging
logging.getLogger("py4j").setLevel(logging.ERROR)

import os
import time
import datetime 
from datetime import timedelta

import requests
from config import *

In [5]:
spark = SparkSession.builder.appName("KafkaStream")\
        .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

In [6]:
schema = StructType([
    StructField("name", StringType(), True),
    StructField("timestart", IntegerType(), True),
    # StructField("price", DoubleType(), True),
    StructField("volume", DoubleType(), True)
    # StructField("num", IntegerType(), True)
])

# time.sleep(10)

In [7]:
# kafka_server = "localhost:9094"
kafka_server = 'localhost:9092'
input_topic = "coin"

In [ ]:
kafka_server = "localhost:9092,localhost:29092,localhost:9999"
input_topic = "coin"

In [8]:
print(kafka_server)

localhost:9092


In [19]:
spark_version = '3.5.1' 
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_2.12:{spark_version} streaming.py'

spark = SparkSession.builder.appName("KafkaStream")\
        .getOrCreate()

spark.sparkContext.setLogLevel('ERROR')

AttributeError: 'SparkSession' object has no attribute 'config'

In [18]:
os.environ['PYSPARK_SUBMIT_ARGS']

'--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 streaming.py'

In [13]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_server) \
  .option("subscribe", input_topic) \
  .option("startingOffsets", "latest") \
  .load() \
  .selectExpr("CAST(value AS STRING)") \
  .select(from_json("value", schema).alias("data")) \
  .select("data.*")

# df = spark \
#   .readStream \
#   .format("kafka") \
#   .option("kafka.bootstrap.servers", kafka_server) \
#   .option("subscribe", input_topic) \
#   .load()

# df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.